# Lab 4-: Mixture Models+Model orden selection 

The goal of this lab session is to study mixture models. In the first part you will code the EM algorithm to estimate the parameters of a GMM given the number of mixed distributions and in the second part you will try different model order selection methods. You will send only one notebook for both parts **(this notebook will be updated with instructions for the second part and the deadline)**.

You have to send the filled notebook named **"L4__familyname1_familyname2.ipynb"** (groups of 2) by email to aml.centralesupelec.2019@gmail.com and put **"AML-L4- "** in the subject. 

We begin with the standard imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

## GMM

A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. After estimation of those parameters we get an estimation of the distribution of our data. For the clustering task, one can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians. 

### First part

Fill in the following class to implement a multivariate GMM:

In [35]:
x = np.random.rand(2, 3, 5)

In [43]:
np.matmul(np.expand_dims(x, -1), np.expand_dims(x, axis = 2))

(2, 3, 5, 5)

In [31]:
np.expand_dims(x, axis = 1).shape

(2, 1, 3)

In [23]:
mu = np.random.rand(2, 3)

In [12]:
from scipy.stats import multivariate_normal

In [25]:
f = np.vectorize(multivariate_normal.pdf)

In [9]:
multivariate_normal.pdf(X, mean=self.mu_, cov=self.Sigma)

In [20]:
x[:,] = np.eye(3)

In [ ]:
class my_GMM():
    
    def __init__(self, k):
        '''
        Parameters:
        k: integer
            number of components
        
        Attributes:
        
        alpha_: np.array
            proportion of components
        mu_: np.array
            array containing means
        Sigma_: np.array
            array cointaining covariance matrix
        cond_prob_: (n, K) np.array
            conditional probabilities for all data points 
        labels_: (n, ) np.array
            labels for data points
        '''
        self.k = k
        self.alpha_ = None
        self.mu_ = None
        self.Sigma_ = None
        self.cond_prob_ = None
        self.labels_ = None
        
    def fit(self, X, initialization = 'random'):
        """ Find the parameters
        that better fit the data
        
        Parameters:
        -----------
        X: (n, p) np.array
            Data matrix
        
        Returns:
        -----
        self
        """
        n, p = X.shape
        k = self.k
        if self.initialization == 'random' :
            self.mu_ = np.random.rand(k, p)
            self.Sigma_ = np.zeros((k, p, p))
            self.Sigma_[:,] = np.eye(p)
            self.alpha_ = np.random.rand(k)
            self.alpha_ = self.alpha_/np.sum(self.alpha_)
        
        def compute_condition_prob_matrix(X, alpha, mu, Sigma):
            '''Compute the conditional probability matrix 
            shape: (n, K)
            '''
            self.cond_prob_ = np.zeros((n, k))
            
        # TODO:
        # initialize the parameters
        # apply sklearn kmeans or randomly initialize them
        convergence = False
        While not(convergence) :
            density = np.zeros((n, k))
            for i in range (k):
                density[:, i] = multivariate_normal.pdf(X, mean=self.mu_[i], cov=self.Sigma[i])
            self.cond_prob_ = self.alpha_.T * density
            self.cond_prob_ = self.cond_prob_ / np.sum(self.cond_prob_, axis = 1)
            
            self.alpha_ = np.mean(self.cond_prob_, axis = 0)
            self.mu_ = (1/self.alpha_) * np.mean( np.expand_dims(X, axis = 1) * self.cond_prob_, axis = 0)
            tmp = np.expand_dims(X, axis=1) - np.expand_dims(self.mu_, axis = 0)
            self.Sigma_ = (1/self.alpha_) * np.sum(p * np.matmul(np.expand_dims(tmp, -1), np.expand_dims(tmp, 2)),
                                                   axis = 0)
            ## RAJOUTER UN CRITERE DE CONVERGENCE 
            
        #     Compute conditional probability matrix
        #     Update parameters
        
        # Update labels_
        
        Return self
        
    def predict(self, X):
        """ Predict labels for X
        
        Parameters:
        -----------
        X: (n, p) np.array
            New data matrix
        
        Returns:
        -----
        label assigment        
        """
        # TODO
        
    def compute_proba(self, X):
        """ Compute probability vector for X
        
        Parameters:
        -----------
        X: (n, p) np.array
            New data matrix
        
        Returns:
        -----
        proba: (n, k) np.array        
        """
        # TODO

Generate your own mixture of Gaussian distributions to test the model, choose parameters so that GMM performs better than K-Means on it. Use `np.random.multivariate_normal`. 

Plot data with colors representing predicted labels and shapes representing real labels.

In [ ]:
# TODO

#### Bonus (not graded): Implement a mixture of asymmetric generalized Gaussians (AGGD)

### Second Part
 
To be updated